#CafChem tools for using Futurehouse's ether0 chatbot, a version of Mistral-Small-24B-Instruct-2501 finetuned for chemistry/medicinal chemistry applications.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MauricioCafiero/CafChem/blob/main/notebooks/Ether0_CafChem.ipynb)

## This notebook allows you to:
- Load either the full or a quantized version of ether0
- perform inference with the model.

## Requirements:
- This notebook will install gguf
- It will install all needed libraries.
- Recommend the v2-8 TPU for memory (25 GB for quantzed, ~100 GB for full) but in either case will use CPU for inference. Inference will take a *long time!*

## Use cases (from the [futurehouse Huggingface page](https://huggingface.co/futurehouse/ether0))

- IUPAC name to SMILES
- Molecular formula (Hill notation) to SMILES, optionally with constraints on functional groups
- Modifying solubilities on given molecules (SMILES) by specific LogS, optionally with constraints about scaffolds/groups/similarity
- Matching pKa to molecules, proposing molecules with a pKa, or modifying molecules to adjust pKa
- Matching scent/smell to molecules and modifying molecules to adjust scent
- Matching human cell receptor binding + mode (e.g., agonist) to molecule or modifying a molecule's binding effect. Trained from EveBio
- ADME properties (e.g., MDDK efflux ratio, LD50)
- GHS classifications (as words, not codes, like "carcinogen"). For example, "modify this molecule to remove acute toxicity."
- Quantitative LD50 in mg/kg
- Proposing 1-step retrosynthesis from likely commercially available reagents
- Predicting a reaction outcome
- General natural language description of a specific molecule to that molecule (inverse molecule captioning)
- Natural product elucidation (formula + organism to SMILES) - e.g, "A molecule with formula C6H12O6 was isolated from Homo sapiens, what could it be?"
- Matching blood-brain barrier permeability (as a class) or modifying

For example, you can ask "Propose a molecule with a pKa of 9.2" or "Modify CCCCC(O)=OH to increase its pKa by about 1 unit." You cannot ask it "What is the pKa of CCCCC(O)=OH?" If you ask it questions that lie significantly beyond those tasks, it can fail. You can combine properties, although we haven't significantly benchmarked this.

## Set-up

### Install libraries

In [1]:
!pip install gguf>=0.10.0

In [2]:
!pip show gguf

Name: gguf
Version: 0.17.1
Summary: Read and write ML models in GGUF for GGML
Home-page: https://ggml.ai
Author: GGML
Author-email: ggml@ggml.ai
License: 
Location: /usr/local/lib/python3.12/dist-packages
Requires: numpy, pyyaml, tqdm
Required-by: 


### Import libraries

In [3]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

/usr/local/lib/python3.12/dist-packages/torch_xla/experimental/gru.py:113: SyntaxWarning: invalid escape sequence '\_'
  * **h_n**: tensor of shape :math:`(D * \text{num\_layers}, H_{out})` or


### Define functions

In [4]:
def make_pipe(ether0_type: str):
  '''
    setup a pipe with either the full ether0 model or the quantized version

    Args:
      ether0_type: either full model or quantized version

    Returns:
      pipe: pipeline object with the chosen model
  '''
  if ether0_type == "full":
    model_id = "futurehouse/ether0"
    pipe = pipeline("text-generation", model=model_id)
  # elif ether0_type == "other":
  #   model_id = "redponike/ether0-GGUF"
  #   filename = "ether0-Q6_K.gguf"

  #   torch_dtype = torch.float32 # could be torch.float16 or torch.bfloat16 too
  #   tokenizer = AutoTokenizer.from_pretrained(model_id, gguf_file=filename)
  #   model = AutoModelForCausalLM.from_pretrained(model_id, gguf_file=filename, torch_dtype=torch_dtype)
  elif ether0_type == "quantized":
    model_id = "DevQuasar/futurehouse.ether0-GGUF"
    filename = "futurehouse.ether0.Q8_0.gguf"

    torch_dtype = torch.float32 # could also be torch.float16 or torch.bfloat16
    tokenizer = AutoTokenizer.from_pretrained(model_id, gguf_file=filename)
    model = AutoModelForCausalLM.from_pretrained(model_id, gguf_file=filename, torch_dtype=torch_dtype)

    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

    print("Model pipeline has been initialized!")

  return pipe

## Inference

In [5]:
quant_pipe = make_pipe("quantized")

futurehouse.ether0.Q8_0.gguf:   0%|          | 0.00/25.1G [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


Converting and de-quantizing GGUF tensors...:   0%|          | 0/363 [00:00<?, ?it/s]

Device set to use cpu


Model pipeline has been initialized!


In [6]:
result = quant_pipe("Propose a molecule with a pKa of 10.1")

print(result[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Propose a molecule with a pKa of 10.19, an H-bond donor, and acceptor, and a benzyl group.

Let's consider benzyl alcohol, which has a pKa around 10.06. Adding a methyl group to the alcohol would make it 2-methoxybenzyl alcohol (pKa ~10.21). However, the user's pKa is 10.19, which is slightly lower. Let's consider a structure with an amide or ester group.

Consider a ketone like benzyl acetone, which has a pKa around 18.93. Adding hydroxyl and amine groups would lower the pKa, but it might not reach 10.19. Consider a substituted benzyl alcohol.

Consider benzylamine derivatives. Benzylamine is C7H9N. Adding a hydroxyl group and another substituent. Consider a pKa around 10.19.

Consider a substituted benzene ring with an amide or ester. For instance, a benzene ring with a methyl ester (COOCH3) and a pKa around 10.19. Adding a hydroxyl group.

Consider a substituted benzene ring with a hydroxyl, an amine, and another group. For instance, a benzene ring with -OH, -NH2, and
